## Feature Engineering House Price
1. Handling Missing Values
2. Temporal Variable
3. Handle Categorical Variables
4. Standarddize the variable

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
import stat

In [2]:
dataset = pd.read_csv("Dataset/Housing_price/train.csv")
dataset.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1025,1026,20,RL,70.0,7700,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2007,WD,Normal,112500
1273,1274,80,RL,124.0,11512,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,5,2008,WD,Normal,177000
698,699,20,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal,138500
1171,1172,20,RL,76.0,9120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,1400,11,2008,WD,Normal,163000
794,795,60,RL,NaN,10832,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2008,WD,Normal,194500


In [5]:
X = dataset.drop(["Id","SalePrice"], axis=1)
y = dataset["SalePrice"]

In [6]:
X.shape, y.shape

((1460, 79), (1460,))

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
X_train.shape, X_test.shape

((1168, 79), (292, 79))

### Handling Missing values

In [20]:
feature_nan = [[feature,dataset[feature].isnull().mean()*100] for feature in dataset.columns if dataset[feature].isnull().sum()>=1]

In [21]:
for feature,val in feature_nan:
    print(f"{feature} missing values in %age is {val}")

LotFrontage missing values in %age is 17.73972602739726
Alley missing values in %age is 93.76712328767123
MasVnrType missing values in %age is 0.547945205479452
MasVnrArea missing values in %age is 0.547945205479452
BsmtQual missing values in %age is 2.5342465753424657
BsmtCond missing values in %age is 2.5342465753424657
BsmtExposure missing values in %age is 2.6027397260273974
BsmtFinType1 missing values in %age is 2.5342465753424657
BsmtFinType2 missing values in %age is 2.6027397260273974
Electrical missing values in %age is 0.0684931506849315
FireplaceQu missing values in %age is 47.26027397260274
GarageType missing values in %age is 5.5479452054794525
GarageYrBlt missing values in %age is 5.5479452054794525
GarageFinish missing values in %age is 5.5479452054794525
GarageQual missing values in %age is 5.5479452054794525
GarageCond missing values in %age is 5.5479452054794525
PoolQC missing values in %age is 99.52054794520548
Fence missing values in %age is 80.75342465753424
MiscFe

In [30]:
# Categorical Features
feature_cat = [feature for feature in dataset.columns if dataset[feature].dtypes=='O']

In [29]:
data = dataset.copy()
def replace_cat_feature(data, cat_feature):
    mode_value = data[feature].mode()
    

Id               False
MSSubClass       False
MSZoning          True
LotFrontage      False
LotArea          False
                 ...  
MoSold           False
YrSold           False
SaleType          True
SaleCondition     True
SalePrice        False
Length: 81, dtype: bool

In [32]:
#Numerical values
feature_num_nan = [feature for feature in dataset.columns if dataset[feature].dtypes!='O' and dataset[feature].isnull().sum()>=1]

In [33]:
feature_num_nan

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [34]:
for feature in feature_num_nan:
    median_values = dataset[feature].median()
    ## Creating a new feature to capture nan values()
    dataset[feature+'nan'] = np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_values,inplace=True)

In [35]:
dataset.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,LotFrontagenan,MasVnrAreanan,GarageYrBltnan
1091,1092,160,FV,24.0,2160,Pave,Pave,Reg,Lvl,AllPub,...,NaN,0,3,2008,WD,Normal,160000,0,0,0
163,164,45,RL,55.0,5500,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,4,2007,WD,Normal,103200,0,0,1
782,783,20,RL,67.0,16285,Pave,NaN,IR2,Lvl,AllPub,...,NaN,0,6,2009,WD,Normal,187100,0,0,0
53,54,20,RL,68.0,50271,Pave,NaN,IR1,Low,AllPub,...,NaN,0,11,2006,WD,Normal,385000,0,0,0
918,919,60,RL,103.0,13125,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,11,2007,WD,Normal,238000,0,0,0


In [37]:
yr_features = [feature for feature in dataset.columns if feature[0:2] == "Yr" or feature[0:2] == "Ye"]